### Matrix generation

In [58]:
import os
import glob
import numpy as np

from csv import reader

In [59]:
os.getcwd()

'E:\\Work\\PhD\\scior\\Scior-Tester\\catalog'

In [20]:
with open('valid_taxonomies_c.txt') as f:
    valid_ac = [line.strip() for line in f]
with open('valid_taxonomies_n.txt') as f:
    valid_an = [line.strip() for line in f]
print(len(valid_ac))
print(len(valid_an))

435
435


In [21]:
taxonomy_length = {}

for valid_taxonomy in valid_an:
    folder_idx = valid_taxonomy.find(os.path.sep)
    for file_name in glob.glob(f"{valid_taxonomy[:folder_idx+1]}data_{valid_taxonomy[folder_idx+1:-4]}.csv"):
        with open(file_name, 'r') as file:
            taxonomy_length[valid_taxonomy] = sum(1 for line in file) - 1

In [22]:
taxonomy_length

{'abrahao2018agriculture-operations\\abrahao2018agriculture-operations_tx001.ttl': 4,
 'abrahao2018agriculture-operations\\abrahao2018agriculture-operations_tx002.ttl': 7,
 'abrahao2018agriculture-operations\\abrahao2018agriculture-operations_tx003.ttl': 6,
 'aguiar2018rdbs-o\\aguiar2018rdbs-o_tx003.ttl': 3,
 'aguiar2018rdbs-o\\aguiar2018rdbs-o_tx004.ttl': 3,
 'aguiar2018rdbs-o\\aguiar2018rdbs-o_tx005.ttl': 3,
 'aguiar2018rdbs-o\\aguiar2018rdbs-o_tx006.ttl': 3,
 'aguiar2019ooco\\aguiar2019ooco_tx001.ttl': 7,
 'aguiar2019ooco\\aguiar2019ooco_tx003.ttl': 3,
 'aguiar2019ooco\\aguiar2019ooco_tx004.ttl': 3,
 'aires2022valuenetworks-geo\\aires2022valuenetworks-geo_tx001.ttl': 4,
 'aires2022valuenetworks-geo\\aires2022valuenetworks-geo_tx002.ttl': 7,
 'aires2022valuenetworks-geo\\aires2022valuenetworks-geo_tx003.ttl': 6,
 'albuquerque2011ontobio\\albuquerque2011ontobio_tx009.ttl': 2,
 'alpinebits2022\\alpinebits2022_tx010.ttl': 7,
 'amaral2019rot\\amaral2019rot_tx002.ttl': 26,
 'amaral2019rot

In [73]:
def is_valid(name, completeness) -> str:
    folder = name.split("\\")[0]
    taxonomy_name = f"{folder}\\{folder}_{name.split('_')[6]}.ttl"
    valid_list = valid_ac if completeness == "ac" else valid_an
    if taxonomy_name in valid_list:
        return taxonomy_name
    return ""

In [74]:
def calculate_matrices(complete_list: list, names: list):
    for name in names:
        num = 15 if name == "class" else 9
        for complete in complete_list:
            matrix = np.zeros((4,num), dtype=float)
            idx = 0
            for percentage in range(2, 10, 2):
                temp = np.zeros((num,num), dtype=float)
                for file_name in glob.glob(f"*\\tt002_{complete}\\results\\{name}_matrix_*_pc0{percentage}0.csv"):
                    taxonomy_name = is_valid(file_name, complete)
                    if taxonomy_name:
                        result = np.array(list(reader(open(file_name, "r"), delimiter=","))).astype('float')
                        result /= taxonomy_length[taxonomy_name]
                        temp += result
                total = sum(sum(temp))
                temp *= 100
                temp /=total
                matrix[idx,:] = matrix[idx,:] + temp[0,:]
                idx += 1
            np.savetxt(f"{name}_matrixes_{complete}.csv", matrix, delimiter=",")

In [75]:
calculate_matrices(["ac", "an"], ["class", "leaves"])